In [1]:
import pandas as pd
import os
import numpy as np

# import glove
# from glove import Corpus
# from mittens import GloVe

import collections
import gc 

import gensim.downloader as api
from gensim.scripts.glove2word2vec import glove2word2vec

import warnings
warnings.filterwarnings('ignore')

In [2]:
# glove_model_twitter = api.load('glove-twitter-25')
glove_model = api.load('glove-wiki-gigaword-300')



[==================================================] 100.0% 376.1/376.1MB downloaded


In [3]:
med7_ner_data = pd.read_pickle("data/new_ner_word_dict.pkl")

In [4]:
def mean(a):
    return sum(a) / len(a)

In [5]:
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_glove_vec = {}
    print("Glove starting..")
    for k,v in data.items():

        # if k == 15219:
        #     print("27586")

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(glove_model[i[0]])
            except:
                avg = []
                num = 0
                temp = []
                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = glove_model[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    t = np.asarray(list(map(mean, zip(*avg))))
                    patient_temp.append(t)
                              
        if len(patient_temp) == 0: continue

        new_glove_vec[k] = patient_temp

    print(len(new_glove_vec))
    pd.to_pickle(new_glove_vec, "data/"+names+"_Glove_dict.pkl")

Glove starting..
21580


In [8]:
ner_fasttext = pd.read_pickle("data/new_ner_fasttext_limited_dict.pkl")

diff = set(new_glove_vec.keys()).difference(set(ner_fasttext))
for i in diff:
    del new_glove_vec[i]


print (len(new_glove_vec), len(ner_fasttext))


pd.to_pickle(new_glove_vec, "data/"+"new_ner"+"_Glove_limited_dict.pkl")


21550 21731
